<a href="https://colab.research.google.com/github/KanonJager/Practica2/blob/Practica-2/Practica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------------------------PRACTICA DOS----------------------------- 


Integrantes del equipo:


Mayo Ruíz Ademir Sisifo 



Ortiz Morales María José 


Sánchez Olvera Nashelly Natasha


Yañez Romero Trianna Montserrat

In [21]:
#Importando librerias
import nltk
nltk.download('stopwords')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
import numpy as np    
from numpy import dot
from numpy.linalg import norm
import scipy as sc
from scipy.spatial import distance
import pandas as pd
import numpy as np
import missingno as msno

stopwords.fileids()
set(stopwords.words('spanish'))
tokenizer = RegexpTokenizer(r'\w+')

path = 'tweets.csv'   #Obtenemos archivo csv con mensajes
df = pd.read_csv(path, encoding='latin', error_bad_lines=False)
#df.head()
#df['tweet'].count()

texts=[]    ### creamos arreglo de tokens 
for i in range(0,df['tweet'].count()):
    texts.append(df['tweet'][i])
#texts

text_token=[]

text_stops= [ [0 for columna in range(0,0)] for fila in range (0,df['tweet'].count())]
for i in range(0,df['tweet'].count()):
    text_token.append(tokenizer.tokenize(texts[i].lower()))

for i in range(0,df['tweet'].count()):
      for word in text_token[i]:
        if word not in stopwords.words('spanish'): text_stops[i].append(word)
#text_token
#print(text_stops)#text_token

##    Creación de la BoW
dicc_texts2={}
for i in range(0,df['tweet'].count()):
  var='tweet'+ str(i+1)
  dicc_texts2.setdefault(var,text_stops[i])

#dicc_texts2

dicc_termns2 = {}

for text in dicc_texts2:
    for word in dicc_texts2[text]:
#        print("EVALUAR:", word, "EN", text)
        if(word in dicc_termns2):#incrementar palabras al diccionario
            dicc_termns2[word] = dicc_termns2[word] + 1
#            print(word, "IN", "dicc_termns")
        elif(word not in dicc_termns2):#agregar palabras al diccionario        
            dicc_termns2[word] = 1
#            print(word, "NOT IN", "dicc_termns")            
print(len(dicc_termns2))
#dicc_termns2

##   Matriz Término Documento (binaria)
matrixA = np.zeros((len(dicc_texts2), len(dicc_termns2)))
#matrixA

i = 0
j = 0

for word_termns2 in dicc_termns2: #dicc_termns todos los términos
    for word_texts2 in dicc_texts2: #dicc_texts todos los textos
#        print("EVALUAR:", word_termns, "EN: ", word_texts)
        if(word_termns2 in dicc_texts2[word_texts2]): #si está
            #print(word_termns2, "IN", word_texts2)
            
            matrixA[j, i] = 1
            
        elif(word_termns2 not in dicc_texts2[word_texts2]): # si no está
            #print(word_termns2, "NOT IN", word_texts2)
            
            matrixA[j, i] = 0
            
            
        #print("se agregó: ", matrixA[j,i], "en: ", j, i)
            
        j = j + 1
        
    j = 0
    i = i + 1

# calculamos la relación de cosenos (similaridad) de los textos y guardamos en un arreglo 
resultA =[]  # vector de resultados

for i in range(0,df['tweet'].count()-1):
    for j in range(i+1, df['tweet'].count()):
        resultA.append(dot(matrixA[i],matrixA[j])/(norm(matrixA[i])*norm(matrixA[j])))

#print(resultA)

#   Matriz Término Documento (frecuencia)
matrixB = np.zeros((len(dicc_texts2), len(dicc_termns2))) # Pre-allocate matrix
#matrix

i = 0
j = 0

for word_termns2 in dicc_termns2: #dicc_termns todos los términos
#    print()
    for word_texts2 in dicc_texts2: #dicc_texts todos los textos
#        print("EVALUAR:", word_termns, "EN: ", word_texts)
        if(word_termns2 in dicc_texts2[word_texts2]): #si está
            #print(word_termns2, "IN", word_texts2)
            
            matrixB[j, i] = dicc_termns2[word_termns2]
            
        elif(word_termns2 not in dicc_texts2[word_texts2]): # si no está
            #print(word_termns2, "NOT IN", word_texts2)
            
            matrixB[j, i] = 0
            
            
        #print("se agregó: ", matrixB[j,i], "en: ", j, i)
            
        j = j + 1
        
    j = 0
    i = i + 1
#matrixB

# calculamos la relación de cosenos (similaridad) y distancia euclidiana (disimilaridad) de los textos y guardamos en un arreglos
result2=[]  # vector de resultados
tweets2=[]  # vector de enfrentamientos
resultados=[]

for i in range(0,df['tweet'].count()-1):
    for j in range(i+1, df['tweet'].count()):
        tweets2.append('tweet '+str(i+1)+' vs '+str(j+1))
        result2.append(dot(matrixB[i],matrixB[j])/(norm(matrixB[i])*norm(matrixB[j])))
        resultados.append(distance.euclidean(matrixB[i],matrixB[j]))

matriz =[]  #creamos una matriz auxiliar que guardará los indices de los mensajes comparados 
k=0
for i in range(0,df['tweet'].count()-1):
    for j in range(i+1, df['tweet'].count()):
        matriz.append([])
        matriz[k].append(k)
        matriz[k].append(i)
        matriz[k].append(j)
        k=k+1
#print(matriz)

#  Imprimimos los resultados
meanSim=sum(resultA)/len(resultA)
print('##########################################################','\n','\n')
print('Los tweets con mayor relación son:','\n')
for i in range(0,len(resultA)):
  if(resultA[i]>meanSim):
    print(tweets2[i],' , similaridad ',resultA[i],'\n')
    print('\t','\t',matriz[i][1]+1,' :',df['tweet'][matriz[i][1]],'\n')
    print('\t','\t',matriz[i][2]+1,' :',df['tweet'][matriz[i][2]],'\n')

mean=sum(resultados)/len(resultados)
#mean
print('##########################################################','\n','\n')
print('Los tweets con menor disimilaridad son:','\n')
for i in range(0,len(resultados)):
  if(resultados[i]<mean):
    print(tweets2[i],' , disimilaridad ',resultados[i],'\n')
    print('\t','\t',matriz[i][1]+1,' :',df['tweet'][matriz[i][1]],'\n')
    print('\t','\t',matriz[i][2]+1,' :',df['tweet'][matriz[i][2]],'\n')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
436
########################################################## 
 

Los tweets con mayor relación son: 

tweet 1 vs 2  , similaridad  0.04999999999999999 

	 	 1  : Se celebran? Chale que conformista para américa es un buen resultado pero para pumas no mucho menos cuando sabes que el equipo jugó bien pero siempre tu portero la va a cagar en cada partido y echara a perder todo como siempre 

	 	 2  : Este empate no se puede celebrar ! Da coraje ! Regalas un gol y t empatan en el último minuto ! Q se va a celebrar ! El america mas parchado q nunca ! Y ni así le ganamos ! Ya son mamadas ! Q se va a celebrar ? 

tweet 1 vs 13  , similaridad  0.04225771273642582 

	 	 1  : Se celebran? Chale que conformista para américa es un buen resultado pero para pumas no mucho menos cuando sabes que el equipo jugó bien pero siempre tu portero la va a cagar en cada partido y echara a per